Testing experiments

In [3]:
from simulate_module import *

import pandas as pd
from pathlib import Path
import numpy as np
np.random.seed(1234)


In [4]:
sigma_setting = {"high_bw": [10, .2],
                "medium_bw": [1, .2],
                "low_bw": [.5, .2]}

In [11]:
args = {
    "n_tasks": 15,
    "conservative": False,
    "target_test_size": 0.4,
    "model_type": "lm",
    "base_output_dir": "test"
}

In [12]:
if args["model_type"] == "lm":
    model_class = lm()
    loss_fn = mse
elif args["model_type"] == "nn":
    model_class = nn()
    loss_fn =  torch.nn.MSELoss()

In [16]:
s = "high_bw"
    # set directory
if args["conservative"]:
    data_path = Path(args["base_output_dir"]) / Path("model_" + args["model_type"] + "/conservative_derived_data")
else:
    data_path = Path(args["base_output_dir"]) / Path("model_" + args["model_type"] + "/derived_data")
data_path = Path(data_path)
working_path = data_path / s
working_path.mkdir(parents = True, exist_ok = True)
# generate data ------------------------------------------------
np.random.seed(1234)
f, betas, zs = random_functions(args["n_tasks"], 6,
                                sigma_between = sigma_setting[s][0],
                                sigma_within = sigma_setting[s][-1])
result = []
for i, fi in enumerate(f):
    x = np.random.uniform(0, 1, 100)
    result.append({
        "task": i,
        "x": x,
        "f": fi(x),
        "y": fi(x) + np.random.normal(0, .1, len(x))
    })
# save data
data_df = pd.concat([pd.DataFrame(r) for r in result])
data_df.to_csv(working_path / "tasks.csv", index = False)
data_df = data_df.reset_index()
betas_df = np.hstack([np.arange(args["n_tasks"])[:, np.newaxis], np.array(zs)[:, np.newaxis], betas])
betas_df = pd.DataFrame(betas_df)
betas_df.columns = ["task", "cluster"] + [f"beta{i}" for i in range(betas.shape[1])]
betas_df.to_csv(working_path / "betas.csv", index = False)
data_dict = data_df.to_dict(orient = "list")

In [18]:
input_data = prepare_input(data_dict,
                               target_task = 5,
                               target_test_size = args["target_test_size"],
                              preprocess = True)
pd.DataFrame.from_dict(input_data["data_dict"]).to_csv(working_path / "tasks_processed.csv",
                                                           index = False)

/Users/miaoxinran/Documents/projects/source_selection22/source_selection/simulation/simulate_module.py:84: RuntimeWarning: invalid value encountered in true_divide
  processed = (raw - raw.min(axis = 0)) / (raw.max(axis = 0) - raw.min(axis = 0))


In [28]:
model = lm()
n_it = 100
batch_size = 8
decay_rate = .5
conservative = False

In [30]:
bandit_selects = [None]
# initialize hyperparameters
alpha = dict.fromkeys(input_data["source_task"], [1])
beta = dict.fromkeys(input_data["source_task"], [1])
pi = dict.fromkeys(input_data["source_task"], [0])

mod = model
val_x, val_y = mod.prepare_data(input_data["X_target_val"], input_data["y_target_val"])

# initialize model from target training data
X_current, y_current = mod.prepare_data(input_data["X_target_train"], input_data["y_target_train"])
mod.fit( X_current, y_current)
l = mod.evaluate(val_x, val_y, loss_fn = loss_fn)
losses = [l]
model_old = mod.model

for t in range(n_it):
    mod = model
    # select bandit
    bandit_current, pi = get_bandit(input_data, alpha, beta,t, pi)
    bandit_selects.append(bandit_current)
    # set training data at this iteration
    X_current, y_current, _ = subset_data(input_data["source_dict"], 
                               key_value = bandit_current,
                               key_name = "task", test_size = 0)
    batch_id = random.choices(list(range(0, len(y_current))), k = batch_size)
    X_current, y_current = X_current[batch_id, :], y_current[batch_id]

    X_current = np.concatenate((X_current, input_data["X_target_val"]), axis = 0)
    y_current = np.concatenate((y_current, input_data["y_target_val"]), axis = 0)
    X_current, y_current = mod.prepare_data(X_current, y_current)
    # train model
    #mod_train = model.fit(X_current, y_current)
    mod.fit(X_current, y_current, loss_fn = loss_fn)
    # combine parameters with previous model
    mod.combine_with_old(model_old, decay_rate = decay_rate)
    # evaluate model
    l = mod.evaluate(val_x, val_y, loss_fn = loss_fn)
    losses += [l]
    model_old = mod.model
    # update bandit parameters
    if conservative:
        thres = 100000
    else:
        thres = avg_loss(bandit_selects, losses, bandit_current)
    alpha, beta = update_hyper_para(alpha, beta, t, losses,
                                    bandit_current,
                                    thres = thres
                                   )
# baseline   
bl = baseline(input_data = input_data, alpha = alpha, beta = beta,
              N = 10, model = model, loss_fn = loss_fn)


In [31]:
weight_dict = {t: alpha[t][-1] / (alpha[t][-1] + beta[t][-1]) for t in input_data["source_task"]}
s = sum(weight_dict.values())
weight_dict = {t: weight_dict[t] / s for t in weight_dict.keys()}


In [32]:
weight_dict

{0: 0.005077891224315845,
 1: 0.17095567121863342,
 2: 0.005077891224315845,
 3: 0.005077891224315845,
 4: 0.005077891224315845,
 6: 0.10257340273118007,
 7: 0.005077891224315845,
 8: 0.36328080133959606,
 9: 0.005077891224315845,
 10: 0.005077891224315845,
 11: 0.005077891224315845,
 12: 0.17095567121863342,
 13: 0.005077891224315845,
 14: 0.14653343247311437}